## Outcome prediction after Chikungunya hospitalization

#### MC853 - Unicamp

- Leandro Henrique Silva Resende – 213437 

- Pietro Grazzioli Golfeto – 223694 

- Yvens Ian Prado Porto – 184031 

escolher uma métrica -> segunda métrica
escolher uma coluna (feature, atributo sensível)
p1 = calcular probabilidade do predito ser 1 dado que real é 1 (morte) e coluna alvo é 1
p2 = calcular probabilidade do predito ser 1 dado que real é 1 (morte) e coluna alvo é 0
calcula métrica p1 - p2 -> x
remover coluna alvo
retreinar o modelo (random forest com melhor hiperparâmetros da validação, só 1 modelo)
calcular novamente p1, p2 e métrica p1 - p2
analisar resultado, provavelmente vai baixar o desempenho e melhorar um pouco essa métrica

In [28]:
# Required Libraries
# We used Python 3.10.12
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import time

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import (
    make_scorer, recall_score, precision_score, fbeta_score,
    f1_score, balanced_accuracy_score, roc_auc_score,
    average_precision_score
)

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [29]:
# Paths to the data (change according to your system)
leandro_path = {
    'X_train_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/X_train.csv',
    'y_train_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/y_train.csv',
    'X_test_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/X_test.csv',
    'y_test_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/y_test.csv',
    'train_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/train.csv',
    'test_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/test.csv',
}

pietro_path = {
    'X_train_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/X_train.csv',
    'y_train_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/y_train.csv',
    'X_test_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/X_test.csv',
    'y_test_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/y_test.csv',
    'train_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/train.csv',
    'test_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/test.csv',
}

In [30]:
# Set the path based on the user
if os.path.isfile(leandro_path['X_train_path']):
    path = leandro_path
elif os.path.isfile(pietro_path['X_train_path']):
    path = pietro_path
else:
    raise Exception('Path not found. Please check the paths in the script.')

# Get CSV files path (modify to match your file path)
train_path = os.path.expanduser(path['train_path'])
test_path = os.path.expanduser(path['test_path'])
X_train_path = os.path.expanduser(path['X_train_path'])
y_train_path = os.path.expanduser(path['y_train_path'])
X_test_path = os.path.expanduser(path['X_test_path'])
y_test_path = os.path.expanduser(path['y_test_path'])

In [31]:
train = pd.read_csv(train_path, low_memory=False)
test = pd.read_csv(test_path, low_memory=False)
X_train = pd.read_csv(X_train_path, low_memory=False)
y_train = pd.read_csv(y_train_path, low_memory=False).squeeze("columns")
X_test = pd.read_csv(X_test_path, low_memory=False)
y_test = pd.read_csv(y_test_path, low_memory=False).squeeze("columns")

In [32]:
X_train.head()

,AGE,GENDER,PREGNANT,FEBRE,MIALGIA,CEFALEIA,EXANTEMA,VOMITO,NAUSEA,DOR_COSTAS,...,REGION_MIDWEST,REGION_SOUTHEAST,REGION_SOUTH,TIME_DIFF_DAYS,TIME,WHITE,BLACK,YELLOW,BROWN,INDIGENOUS
0,10.0,0,0.0,1,1,1,1,0,1,1,...,0,0,0,1,26,0,0,0,1,0
1,29.0,1,0.0,1,1,1,1,0,1,1,...,0,0,0,5,25,0,0,0,1,0
2,11.0,1,0.0,1,1,1,0,0,0,0,...,0,0,0,2,37,0,0,0,1,0
3,5.0,1,0.0,1,0,1,0,0,1,0,...,0,0,0,31,38,0,0,0,1,0
4,11.0,1,0.0,1,1,1,0,0,0,0,...,0,0,0,3,32,0,0,0,1,0


In [33]:
X_train.columns

Index(['AGE', 'GENDER', 'PREGNANT', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA',
       'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA',
       'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'DIABETES',
       'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT',
       'AUTO_IMUNE', 'CONFIRMED_CASE', 'CRITERIO', 'REGION_NORTH',
       'REGION_NORTHEAST', 'REGION_MIDWEST', 'REGION_SOUTHEAST',
       'REGION_SOUTH', 'TIME_DIFF_DAYS', 'TIME', 'WHITE', 'BLACK', 'YELLOW',
       'BROWN', 'INDIGENOUS'],
      dtype='object')

In [47]:
old = train[train['AGE'] > 60]
print(f"Old age count: {len(old)}")
print(f"Old age proportion: {len(old) * 100 / len(train):.2f}%")

Old age count: 3366
Old age proportion: 15.49%


In [48]:
woman = train[X_train['GENDER'] == 1]
print(f"Woman count: {len(woman)}")
print(f"Woman proportion: {len(woman) * 100 / len(train):.2f}%")

man = train[X_train['GENDER'] == 0]
print(f"Man count: {len(man)}")
print(f"Man proportion: {len(man) * 100 / len(train):.2f}%")

Woman count: 11397
Woman proportion: 52.46%
Man count: 10329
Man proportion: 47.54%


In [50]:
southern_regions = X_train[(X_train['REGION_SOUTH'] == 1) | (X_train['REGION_SOUTHEAST'] == 1)]
print(f"Southern region count: {len(southern_regions)}")
print(f"Southern region proportion: {len(southern_regions) * 100 / len(X_train):.2f}%")

Southern region count: 8193
Southern region proportion: 37.71%


In [44]:
white = X_train[X_train['WHITE'] == 1]
print(f"White count: {len(white)}")
print(f"White proportion: {len(white) * 100 / len(X_train):.2f}%\n")

non_white = X_train[X_train['WHITE'] == 0]
print(f"Non-white count: {len(non_white)}")
print(f"Non-white proportion: {len(non_white) * 100 / len(X_train):.2f}%")

White count: 5568
White proportion: 25.63%

Non-white count: 16158
Non-white proportion: 74.37%


In [45]:
white_evolution = train[(train['WHITE'] == 1) & (train['EVOLUCAO'] == 1)]
print(f"White evolution count: {len(white_evolution)}")
print(f"White evolution proportion: {len(white_evolution) * 100 / len(white):.2f}%\n")

non_white_evolution = train[(train['WHITE'] == 0) & (train['EVOLUCAO'] == 1)]
print(f"Non-White evolution count: {len(non_white_evolution)}")
print(f"Non-White evolution proportion: {len(non_white_evolution) * 100 / len(non_white):.2f}%")

White evolution count: 451
White evolution proportion: 8.10%

Non-White evolution count: 1135
Non-White evolution proportion: 7.02%
